# PPO for Portfolio Management
This tutorial is to demonstrate an example of using PPO to do portfolio management

## Step1: Import Packages

In [7]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.utils import plot
from trademaster.utils import set_seed
set_seed(2023)

2023-05-16 20:17:26,018	INFO worker.py:973 -- Calling ray.init() again after it has already been called.
[autoreload of trademaster.trainers.portfolio_management.reinforce_trainer failed: Traceback (most recent call last):
  File "/Users/louison/opt/anaconda3/envs/TradeMaster/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/Users/louison/opt/anaconda3/envs/TradeMaster/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
  File "/Users/louison/opt/anaconda3/envs/TradeMaster/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/louison/PersonalTrade/trademaster/trainers/portfolio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Step2: Import Configs

In [12]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
# parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_sse500_sarl_sarl_adam_mse.py"),
# parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_exchange_ppo_ppo_adam_mse.py"),
parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_sz50_sarl_sarl_adam_mse.py"),
# parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_dj30_deeptrader_deeptrader_adam_mse.py"),
# parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_sz50_reinforce_reinforce_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)
print(cfg)

Config (path: /Users/louison/PersonalTrade/configs/portfolio_management/portfolio_management_sz50_reinforce_reinforce_adam_mse.py): {'data': {'type': 'PortfolioManagementDataset', 'data_path': 'data/portfolio_management/sz50', 'train_path': 'data/portfolio_management/sz50/train.csv', 'valid_path': 'data/portfolio_management/sz50/valid.csv', 'test_path': 'data/portfolio_management/sz50/test.csv', 'tech_indicator_list': ['zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10', 'zd_15', 'zd_20', 'zd_25', 'zd_30'], 'length_day': 10, 'initial_amount': 100000, 'transaction_cost_pct': 0.001, 'test_dynamic_path': 'data/portfolio_management/sz50/test.csv'}, 'environment': {'type': 'PortfolioManagementDeepTraderEnvironment'}, 'trainer': {'type': 'PortfolioManagementREINFORCETrainer', 'agent_name': 'pg', 'if_remove': False, 'configs': {'framework': 'tf2', 'num_workers': 0, 'rollout_fragment_length': 500, 'explore': True}, 'work_dir': 'work_dir/portfolio_management_sz50_deeptrader_pg_adam_ms

## Step3: Build Dataset

In [13]:
dataset = build_dataset(cfg)
print(vars(dataset))


{'kwargs': {'data_path': 'data/portfolio_management/sz50', 'train_path': 'data/portfolio_management/sz50/train.csv', 'valid_path': 'data/portfolio_management/sz50/valid.csv', 'test_path': 'data/portfolio_management/sz50/test.csv', 'tech_indicator_list': ['zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10', 'zd_15', 'zd_20', 'zd_25', 'zd_30'], 'length_day': 10, 'initial_amount': 100000, 'transaction_cost_pct': 0.001, 'test_dynamic_path': 'data/portfolio_management/sz50/test.csv'}, 'data_path': '/Users/louison/PersonalTrade/data/portfolio_management/sz50', 'train_path': '/Users/louison/PersonalTrade/data/portfolio_management/sz50/train.csv', 'valid_path': '/Users/louison/PersonalTrade/data/portfolio_management/sz50/valid.csv', 'test_path': '/Users/louison/PersonalTrade/data/portfolio_management/sz50/test.csv', 'test_dynamic_path': '/Users/louison/PersonalTrade/data/portfolio_management/sz50/test.csv', 'tech_indicator_list': ['zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5',

## Step4: Build Trainer

In [14]:
from ray.tune.registry import register_env
import ray
from trademaster.environments.portfolio_management.environment import PortfolioManagementEnvironment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
work_dir = os.path.join(ROOT, cfg.trainer.work_dir)
ray.init(ignore_reinit_error=True)
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
cfg.dump(osp.join(work_dir, osp.basename(args.config)))

try:
    del trainer
except:
    pass
trainer = build_trainer(cfg, default_args=dict(dataset=dataset, device = device))
print(vars(trainer))

2023-05-16 20:20:46,803	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


| Arguments Keep work_dir: /Users/louison/PersonalTrade/work_dir/portfolio_management_sz50_deeptrader_pg_adam_mse
{'device': device(type='cpu'), 'configs': {'framework': 'tf2', 'num_workers': 0, 'rollout_fragment_length': 500, 'explore': True, 'env': 'portfolio_management', 'env_config': {'dataset': <trademaster.datasets.portfolio_management.dataset.PortfolioManagementDataset object at 0x1f94aa9d0>, 'task': 'train', 'device': device(type='cpu')}}, 'agent_name': 'pg', 'epochs': 1000, 'dataset': <trademaster.datasets.portfolio_management.dataset.PortfolioManagementDataset object at 0x1f94aa9d0>, 'work_dir': '/Users/louison/PersonalTrade/work_dir/portfolio_management_sz50_deeptrader_pg_adam_mse', 'seeds_list': (12345,), 'random_seed': 12345, 'if_remove': False, 'num_threads': 8, 'trainer_name': <class 'ray.rllib.agents.pg.pg.PGTrainer'>, 'checkpoints_path': '/Users/louison/PersonalTrade/work_dir/portfolio_management_sz50_deeptrader_pg_adam_mse/checkpoints'}


## Step5: Train, Valid and Test

In [15]:
trainer.train_and_valid()

2023-05-16 20:20:48,720	WARNING deprecation.py:46 -- DeprecationWarning: `ray.rllib.agents.pg.default_config::DEFAULT_CONFIG` has been deprecated. Use `ray.rllib.agents.pg.pg.PGConfig(...)` instead. This will raise an error in the future!
2023-05-16 20:20:49,365	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-05-16 20:20:49,622	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Train Episode: [1/1000]
Valid Episode: [1/1000]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
|  27.573871%  |   1.090588  | 1.372496%  |  13.292012%  |
+--------------+-------------+------------+--------------+
Valid Episode Reward Sum: 25580.296313
Train Episode: [2/1000]
Valid Episode: [2/1000]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
|  27.878549%  |   1.101326  | 1.370761%  |  13.240382%  |
+--------------+-------------+------------+--------------+
Valid Episode Reward Sum: 25892.219375
Train Episode: [3/1000]
Valid Episode: [3/1000]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
|  27.81726

KeyboardInterrupt: 

rllib uses distributed training stategy and therefore bad result migh occurs during training

In [ ]:
import ray
from ray.tune.registry import register_env
from trademaster.environments.portfolio_management.environment import PortfolioManagementEnvironment
def env_creator(env_name):
    if env_name == 'portfolio_management':
        env = PortfolioManagementEnvironment
    else:
        raise NotImplementedError
    return env
# ray.init(ignore_reinit_error=True)
# register_env("portfolio_management", lambda config: env_creator("portfolio_management")(config))
trainer.test();

2023-05-12 23:49:47,947	WARNING deprecation.py:46 -- DeprecationWarning: `config['prioritized_replay']` has been deprecated. Replay prioritization specified at new location config['replay_buffer_config']['prioritized_replay'] will be overwritten. This will raise an error in the future!
2023-05-12 23:49:47,949	WARNING deprecation.py:46 -- DeprecationWarning: `config[prioritized_replay_alpha]` has been deprecated. config['replay_buffer_config'][prioritized_replay_alpha] should be used for Q-Learning algorithms. Ignore this warning if you are not using a Q-Learning algorithm and still provide prioritized_replay_alpha. This will raise an error in the future!
2023-05-12 23:49:47,950	WARNING deprecation.py:46 -- DeprecationWarning: `config[prioritized_replay_beta]` has been deprecated. config['replay_buffer_config'][prioritized_replay_beta] should be used for Q-Learning algorithms. Ignore this warning if you are not using a Q-Learning algorithm and still provide prioritized_replay_beta. This

EnvError: The env string you provided ('portfolio_management_sarl') is:
a) Not a supported/installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For VizDoom support: Install VizDoom
   (https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md) and
   `pip install vizdoomgym`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.env.repeat_after_me_env.RepeatAfterMeEnv`


In [ ]:
plot(trainer.test_environment.save_asset_memory(),alg="sarl")

AttributeError: 'PortfolioManagementSARLTrainer' object has no attribute 'test_environment'